In [ ]:
from data_loader import PykrxDataLoader
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from pykrx import stock

stock.get_market_cap('2023-01-01')

In [ ]:
from pykrx import stock
import pandas as pd

start_year = 2005
end_year = 2024

# 결과를 저장할 리스트 생성
records = []

for year in range(start_year, end_year + 1):
    date = f"{year}-12-31"  # 연말 기준 (휴장일일 경우 실제 거래일로 조정 필요)
    df = stock.get_market_cap_by_ticker(date, market='KOSPI')
    # 시가총액 상위 200개 주식코드 추출
    top200_codes = df.sort_values(by='시가총액', ascending=False).head(200).index.tolist()
    # 각 코드에 대해 연도와 함께 리스트에 저장
    for code in top200_codes:
        records.append({'연도': year, '주식코드': code})

# 리스트를 데이터프레임으로 변환
result_df = pd.DataFrame(records)

# 결과 확인
print(result_df)


In [ ]:
frequency = result_df['주식코드'].value_counts()
frequency_df = frequency.reset_index()
frequency_df.columns = ['주식코드', '빈도수']
code_df = frequency_df[['주식코드']]


In [ ]:
start_year = 2004
end_year = 2024

# 결과를 저장할 리스트
records = []

for code in code_df['주식코드']:
    for year in range(start_year, end_year + 1):
        date = f"{year}1231"
        try:
            df = stock.get_market_ohlcv_by_date(date, date, code)
            if not df.empty:
                close_price = df['종가'].iloc[0]
            else:
                close_price = None
        except:
            close_price = None
        records.append({'주식코드': code, '연도': year, '종가': close_price})

# 리스트를 데이터프레임으로 변환
price_df = pd.DataFrame(records)

print(price_df)

In [ ]:
start_year = 2004
end_year = 2024

records = []

for code in code_df['주식코드']:
    # 연도별(연말 기준) OHLCV 데이터 한 번에 가져오기
    df = stock.get_market_ohlcv_by_date(
        f"{start_year}0101", f"{end_year}1231", code, freq='y'
    )
    if not df.empty:
        for date, row in df.iterrows():
            year = date.year
            # 연도 범위 내 데이터만 기록
            if start_year <= year <= end_year:
                records.append({'주식코드': code, '연도': year, '종가': row['종가']})

# long-form 데이터프레임 생성
price_df = pd.DataFrame(records)

print(price_df)

In [ ]:
# 필수 연도 범위 생성 (2004~2024)
required_years = set(range(2004, 2025))

# 주식코드별 연도 존재 여부 체크
complete_stocks = price_df.groupby('주식코드')['연도'].agg(
    lambda x: required_years.issubset(set(x))
)

# 결과 필터링
complete_codes = complete_stocks[complete_stocks].index.tolist()

print("누락 없는 주식코드:", complete_codes)

In [ ]:
pd.DataFrame(price_df[['연도']].value_counts()).sort_values(by='연도')

## 재무지표

In [ ]:
ts_data = pd.read_excel("ts2000.xlsx", dtype={'거래소코드': str})
ts_data

In [ ]:
ts_df = ts_data[['거래소코드','부가가치(IFRS)(백만원)']]

In [ ]:
ts_df['연도'] = ts_data['회계년도'].str.split('/').str[0]

In [ ]:
ts_df = ts_df.sort_values(by=['거래소코드', '연도']).reset_index(drop=True)

# 거래소코드별로 자본집약도의 연도별 변화율(전년 대비 증감률) 계산
ts_df['부가가치_변화율'] = ts_df.groupby('거래소코드')['부가가치(IFRS)(백만원)'].pct_change()

ts_df